###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

In [ ]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import html
import numpy as np

## Caso 1: Scraping de un catálogo: Labirratorium

In [ ]:
URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page='

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [ ]:
# La web tiene 80 páginas con 12 cervezas listadas en cada página.

Hacemos la consulta (request) y creamos la SOPA inicial:

In [ ]:
r = requests.get(url)
soup = bs("", 'lxml')

In [ ]:
# Guardamos lista de cervezas
cervezas_grid = soup.find_all(class_="")

In [ ]:
print(cervezas_grid)

In [ ]:
# necesitamos acceder a cada una de las cervezas del grid:
lista_URLs = []
for cerveza in cervezas_grid:
    URL_cerveza = cerveza.find('')['']
    lista_URLs.append(URL_cerveza)

lista_URLs

In [ ]:
# Hacemos un nuevo request para la primera cerveza: 
r = requests.get(lista_URLs)

# Creamos una sopa específica con la info de cada cerveza
soup_cerveza = bs("", "lxml")

In [ ]:
# Nombre
name = soup_cerveza.find(class_="")
print(name)

In [ ]:
# Precio
price = soup_cerveza.find(class_="current-price").find
print(price)

In [ ]:
# Descripcion corta
try: 
    descr_short = soup_cerveza.find(class_="description-short")
except:
    descr_short = None

print(descr_short)

In [ ]:
# Descripción larga
descr_full_list = []
descr_full_p = soup_cerveza.find(class_="product-description")
for sentence in descr_full_p:
    descr_full_list.append(sentence.text)

descr_full = "\n".join(descr_full_list)

print(descr_full)

In [ ]:
# Imagen
image = soup_cerveza.find(class_="js-qv-product-cover img-fluid")
print(image)

In [ ]:
 # Brand

try:
    brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')
except:
    brand = None
print(brand)

In [ ]:
# Código de barras
try:
    barcode = soup_cerveza.find(class_='product-reference')
except:
    barcode = None
print(barcode)

In [ ]:
# Features
features_dicc = {}

features = soup_cerveza.find(class_="data-sheet")

try:
    features_dt = features.find_all("dt")
    features_dd = features.find_all("dd")
    for feature, value in zip(features_dt, features_dd):
        print(feature.text, value.text)
        features_dicc[feature.text] = value.text
except:
    features_dicc = {}

In [ ]:
# Creamos un Id único que os permita diferenciar cada entrada en la BBDD
counter = 1
id_cerv = "lbt_" + str(counter)

Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

In [ ]:
# Agregamos a una lista
lista_cervezas = []

lista_cervezas.append([
    ("id", id_cerv),
    ("name", name),
    ('price', price),
    ('descr_short', descr_short),
    ('descr_full', descr_full),
    ('image', image),
    ('brand', brand),
    ('barcode', barcode),
    ('features', features_dicc)
])

lista_cervezas

### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

Construir un DataFrame